# Generative Adversarial Nets

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(7).png?raw=true" width=800 height=600>

## Features

### Adversarial nets

<img src='https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_23110_(1).png?raw=true' width=600 height=150>

> * $x$ : Data
> * $p_g$ : Generator's distribution
> * $p_z(z)$ : Prior on input noise variables.
> * $G(z;\theta_g)$ : Data space where $G$ is a differentiable function represented by a multilayer perceptron with parameters $\theta_g$.
> * $D(x;\theta_d)$ : Second multilayer perceptron that outputs a single scalar.
> * $D(x)$ : Represents the probability that $x$ came from the data rather than $p_g$.
> * Train $D$ to maximize the problbility of assigning the correct label to both training examples and samples from $G$.
> * Train $G$ to minimize $log(1-D(G(z)))$.
> * $D$ and $G$ play the following two-player minimax game with value function $V(G, D)$.

<img src='https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_220110(2).png?raw=true' width=900 height=500>

* The training criterion allows one to recover the data generation distribution as $G$ and $D$ are given enough capacity, i.e., in the non-oarametric limit.
* Alternated between $k$ steps of optimizing $D$ and one step of optimizing $G$.
* After several steps of training, if $G$ and $D$ have enough capacity, they will reach a point at which both cannot improve becaues $p_g = p_{data}$.

### Theoretical  Results

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(1).png?raw=true" width=600 height=150>

> * Sample minibatch of $m$ noise samples {$z^{(1)}, z^{(2)},...,z^{(m)}$} from noise prior p_g(z).
> * Sample minibatch of $m$ examples {$z^{(1)}, z^{(2)},...,z^{(m)}$} from data generating distribution $p_{data}(x)$

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(2).png?raw=true" width=300 height=150>

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(3).png?raw=true" width=600 height>

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(4).png?raw=true" width=600 height=150>

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(5).png?raw=true" width=600 height=150>

<img src="https://github.com/drmoon-1st/Paper-Implementation/blob/main/Generative%20Adversarial%20Nets/img/screenshot_230111(6).png?raw=true" width=350 height=150>

## Other Concept

### Parametric model and Non-parametric model

* Parametric model : The model has a fixed number of parameters.
    * Fast and easy to understand model.
    * Low flexibility
* Non-parametric model : The number of parameters grow with the amount of training data.
    * Flexible
    * Slow and hard to understand model.

## Source Code

## Reference

Paper : https://arxiv.org/pdf/1406.2661v1.pdf